In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import json
import numpy as np
import matplotlib.pyplot as plt 
import rasterio
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import cv2
import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install --no-index --no-deps /kaggle/input/hubmap-tools/accelerate-0.21.0-py3-none-any.whl

Processing /kaggle/input/hubmap-tools/accelerate-0.21.0-py3-none-any.whl
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.20.3
    Uninstalling accelerate-0.20.3:
      Successfully uninstalled accelerate-0.20.3


In [3]:
!pip install --no-index --no-deps /kaggle/input/hubmap-tools/smp_pycocotools/wheels/pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl


Processing /kaggle/input/hubmap-tools/smp_pycocotools/wheels/pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl


In [4]:
import torch
from transformers import Mask2FormerImageProcessor#, Mask2FormerConfig
type_dict = {0:'blood_vessel', 1:'glomerulus', 2:'unsure'}


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [5]:
import ast
class CustomDataset_instances:
    def __init__(self, df, transform):
        self.df = df
        self.processor = Mask2FormerImageProcessor.from_pretrained("/kaggle/input/mask2former-swin-tiny-cityscapes-semantic",
                                                      reduce_labels=True,
                                                      ignore_index=255,
                                                      do_resize=False, 
                                                      do_rescale=False, 
                                                      do_normalize=False)
        self.transform = transform
    def __getitem__(self, index):
        # Load image
        item = self.df.iloc[index]
        image = rasterio.open(item.path).read()
        type_ = ast.literal_eval(item['structure_types'])
        coords = ast.literal_eval(item['coordinates'])
#         image = image.astype(float) / 255.0  # Normalize image to [0, 1]
        image_shape = image.shape[1:]
        mask = np.zeros(image_shape, dtype=np.uint8)
        pts_array = []
        obj_cnt = 1
        type_dict = {'blood_vessel':0, 'glomerulus':1, 'unsure':2}
        inst2cls = {0:0}
        for typ, coord in zip(type_, coords):
            if typ == 'unsure':
                continue
            else:
                pts = np.array(coord).reshape((-1, 1, 2)).astype(np.int32)
                cv2.fillPoly(mask, [pts], obj_cnt)
                inst2cls[obj_cnt] = type_dict[typ]+1
                obj_cnt+=1
        if self.transform is not None:
            transformed = self.transform(image=image.transpose(1,2,0), mask=mask)
            image, mask = transformed['image'], transformed['mask']
            image = image.transpose(2,0,1)
        if len(inst2cls.keys())==1:
            # Some image does not have annotation (all ignored)
            inputs = self.processor([image], return_tensors="pt")
            inputs = {k:v.squeeze() for k,v in inputs.items()}
            inputs["class_labels"] = torch.tensor([3])
            inputs["mask_labels"] = torch.zeros((0, inputs["pixel_values"].shape[-2], inputs["pixel_values"].shape[-1]))
            return inputs
        inputs = self.processor([image], [mask], instance_id_to_semantic_id=inst2cls, return_tensors="pt", reduce_labels=True)
        inputs = {k: v.squeeze() if isinstance(v, torch.Tensor) else v[0] for k,v in inputs.items()}
#         print(inputs['class_labels'])
        
        return inputs

    def __len__(self):
        return len(self.df)


In [6]:
import albumentations as A

ADE_MEAN = np.array([123.675, 116.280, 103.530]) / 255
ADE_STD = np.array([58.395, 57.120, 57.375]) / 255

# note that you can include more fancy data augmentation methods here
train_transform = A.Compose([
    A.Normalize(mean=ADE_MEAN, std=ADE_STD),
])

In [7]:
def collate_fn(batch):
    items = torch.stack([example["item_id"] for example in batch])
    pixel_values = torch.stack([example["pixel_values"] for example in batch])
    pixel_mask = torch.stack([example["pixel_mask"] for example in batch])
    class_labels = [example["class_labels"] for example in batch]
    mask_labels = [example["mask_labels"] for example in batch]
    return {"pixel_values": pixel_values, "pixel_mask": pixel_mask, "class_labels": class_labels, "mask_labels": mask_labels}


In [8]:
train_gr = pd.read_csv("/kaggle/input/hubmap-train-test/train_grouped.csv")
val_gr = pd.read_csv("/kaggle/input/hubmap-train-test/val_grouped.csv")


In [9]:
train_dataset = CustomDataset_instances(train_gr, train_transform)
val_dataset = CustomDataset_instances(val_gr, train_transform)

In [10]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

from pytorch_lightning.callbacks import LearningRateMonitor
lr_monitor = LearningRateMonitor(logging_interval='step')

In [11]:
import base64
import numpy as np
from pycocotools import _mask as coco_mask
import typing as t
import zlib


def encode_binary_mask(mask: np.ndarray, conf) -> t.Text:
    """Converts a binary mask into OID challenge encoding ascii text."""

    # check input mask --
    if mask.dtype != np.bool:
        raise ValueError(
            "encode_binary_mask expects a binary mask, received dtype == %s" %
            mask.dtype)

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        raise ValueError(
            "encode_binary_mask expects a 2d mask, received shape == %s" %
            mask.shape)

    # convert input mask to expected COCO API input --
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

    # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

    # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return f"0 {conf:.2f} {base64_str.decode('utf-8')}"

In [12]:
class Mask2Former_pred(pl.LightningModule):
    def __init__(self):
        super().__init__()
        from transformers import Mask2FormerForUniversalSegmentation
        self.type_dict = {0:'blood_vessel', 1:'glomerulus', 2:'unsure'}
        self.model = Mask2FormerForUniversalSegmentation.from_pretrained("/kaggle/input/mask2former-swin-tiny-cityscapes-semantic", 
                                                            id2label=self.type_dict,
                                                            ignore_mismatched_sizes=True)
        self.processor = Mask2FormerImageProcessor.from_pretrained("/kaggle/input/mask2former-swin-tiny-cityscapes-semantic",
                                                      reduce_labels=True,
                                                      ignore_index=255,
                                                      do_resize=False, 
                                                      do_rescale=False, 
                                                      do_normalize=False)
        self.learning_rate = 5e-5

    def forward(self, pixel_values, mask_labels, class_labels):
        return self.model(pixel_values, mask_labels, class_labels)

    def training_step(self, batch, batch_idx):
        outputs = self(
            pixel_values=batch["pixel_values"],
            mask_labels=[labels for labels in batch["mask_labels"]],
            class_labels=[labels for labels in batch["class_labels"]],
        )
        loss = outputs.loss
        self.log('train_loss', loss, on_step=True, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        outputs = self(
            pixel_values=batch["pixel_values"],
            mask_labels=[labels for labels in batch["mask_labels"]],
            class_labels=[labels for labels in batch["class_labels"]],
        )
        val_loss = outputs.loss
        self.log('val_loss', val_loss, on_step=False, on_epoch=True)
        
    def predict_step(self, batch, batch_idx):
#         print(batch)
        outputs = self.model(
            pixel_values=batch["pixel_values"],
        )
        pred_instance_map = self.processor.post_process_instance_segmentation(
                            outputs, target_sizes=[(512,512)], return_binary_maps=True

                            )[0]

        pred_str = []
        for d in pred_instance_map['segments_info']:
            if d['label_id']==0:
                mask = pred_instance_map['segmentation'][d['id']].detach().cpu().numpy().astype(bool)
                pred_enc = encode_binary_mask(mask, d['score'])

                pred_str.append(pred_enc)
        return batch['item_id'][0], 512, 512, " ".join(pred_str) 

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate, weight_decay=0.2)

        lr_scheduler_cont = {
            'scheduler':  torch.optim.lr_scheduler.ReduceLROnPlateau(
                                                                        optimizer,
                                                                        mode='min',
                                                                        factor=0.8,
                                                                        patience=50,
                                                                        threshold=1e-2,
                                                                        min_lr=1e-8,
                                                                        verbose=True
                                                                    ),
            'name': 'learning_rate',
            'monitor' : 'train_loss',
            "interval" : "step" 
        }
        return optimizer



  

In [13]:
import ast
class CustomDataset_pred:
    def __init__(self, df, transform):
        self.df = df
        self.processor = Mask2FormerImageProcessor.from_pretrained("/kaggle/input/mask2former-swin-tiny-cityscapes-semantic",
                                                      reduce_labels=True,
                                                      ignore_index=255,
                                                      do_resize=False, 
                                                      do_rescale=False, 
                                                      do_normalize=False)
        self.transform = transform
    def __getitem__(self, index):
        # Load image
        item = self.df[index]
        image = rasterio.open(item).read()
        
        if self.transform is not None:
            transformed = self.transform(image=image.transpose(1,2,0))
            image= transformed['image']
            image = image.transpose(2,0,1)
            
        inputs = self.processor([image], return_tensors="pt",reduce_labels=True)
        inputs = {k: v.squeeze() if isinstance(v, torch.Tensor) else v[0] for k,v in inputs.items()}
        inputs['item_id'] = item.split("/")[-1].split('.')[0]
        return inputs

    def __len__(self):
        return len(self.df)


In [14]:
import glob


In [15]:
# glob.glob('/kaggle/input/hubmap-hacking-the-human-vasculature/test/*') val_gr['path'].tolist()
pred_dataset = CustomDataset_pred(glob.glob('/kaggle/input/hubmap-hacking-the-human-vasculature/test/*'), train_transform)
samp_dataloader = torch.utils.data.DataLoader(pred_dataset, batch_size=1)


In [16]:
model_pred = Mask2Former_pred.load_from_checkpoint("/kaggle/input/hubmap-model-14epch/epoch14-step2460.ckpt", map_location='cpu')

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/utilities/migration/utils.py:51: PossibleUserWarning: The loaded checkpoint was produced with Lightning v2.0.5, which is newer than your current Lightning version: v2.0.4
  rank_zero_warn(
Some weights of Mask2FormerForUniversalSegmentation were not initialized from the model checkpoint at /kaggle/input/mask2former-swin-tiny-cityscapes-semantic and are newly initialized because the shapes did not match:
- class_predictor.weight: found shape torch.Size([20, 256]) in the checkpoint and torch.Size([4, 256]) in the model instantiated
- class_predictor.bias: found shape torch.Size([20]) in the checkpoint and torch.Size([4]) in the model instantiated
- criterion.empty_weight: found shape torch.Size([20]) in the checkpoint and torch.Size([4]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
trainer_pred = pl.Trainer(devices='auto')  # Set the appropriate number of GPUs

res = trainer_pred.predict(model_pred, samp_dataloader)

Predicting: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/tmp/ipykernel_25/2411595467.py:12: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if mask.dtype != np.bool:


In [18]:
submission = pd.DataFrame(res, columns =["id","height","width","prediction_string"])

In [19]:
submission.to_csv('submission.csv',index=False)

In [20]:
# pred_maps = []
# for re in tqdm(res):
#     pred_instance_map = processor.post_process_instance_segmentation(
#         re, target_sizes=[(512,512)], return_binary_maps=True

#     )[0]

#     pred_str = []
#     for d in pred_instance_map['segments_info']:
#         if d['label_id']==0:
#             mask = pred_instance_map['segmentation'][d['id']].numpy().astype(bool)
#             pred_enc = encode_binary_mask(mask, d['score'])

#             pred_str.append(pred_enc)


In [21]:
# img= cv2.cvtColor(cv2.imread('/kaggle/input/hubmap-hacking-the-human-vasculature/test/72e40acccadf.tif'), cv2.COLOR_BGR2RGB) #return np array of pixels 
# plt.figure()
# plt.imshow(img)
# plt.title("Image")
# plt.show()

# predicted_semantic_map = processor.post_process_instance_segmentation(outputs, target_sizes=[(512,512)])[0]
# color_segmentation_map = pred_instance_map['segmentation']
####
# for pred_instance_map in pred_maps:
#     overlay = np.zeros([512,512])
#     pred_str = []
#     for d in pred_instance_map['segments_info']:
#         if d['label_id']==0:
#             mask = pred_instance_map['segmentation'][d['id']].numpy().astype(bool)
#             pred_enc = encode_binary_mask(mask, d['score'])
# #             print(pred_enc)
#             overlay+=pred_instance_map['segmentation'][d['id']].numpy()

#         # plt.imshow(img.reshape(512,512,3))
#             plt.imshow(overlay, alpha=0.6)
#             pred_str.append(pred_enc)
#     break
####
#     print(" ".join(pred_str))